In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch

%matplotlib inline

In [3]:
def load_data():
    name_getter = lambda x: x != 3
    names = pd.read_table(
        "basin_dataset_public_v1p2/basin_mean_forcing/daymet/01/01013500_lump_cida_forcing_leap.txt",
        skiprows=name_getter,
        sep=" ",
    ).columns.to_numpy()
    header = np.zeros(len(names) - 3, dtype=object)
    header[0] = "Time"
    header[1:] = names[4:]
    del names
    df = pd.read_table(
        "basin_dataset_public_v1p2/basin_mean_forcing/daymet/01/01013500_lump_cida_forcing_leap.txt",
        names=header,
        skiprows=4,
    )
    df["Time"] = pd.to_datetime(df["Time"])
    df["Time"] = (df["Time"] - df["Time"][0]).dt.days
    streamflow = pd.read_table(
        "basin_dataset_public_v1p2/usgs_streamflow/01/01013500_streamflow_qc.txt",
        header=None,
        sep="\s+",
    ).astype("str")
    del streamflow[0]
    for i in range(len(streamflow[1])):
        streamflow[1][i] += " " + streamflow[2][i] + " " + streamflow[3][i]
    streamflow[1] = pd.to_datetime(streamflow[1])
    del streamflow[2]
    del streamflow[3]
    streamflow[1] = (streamflow[1] - streamflow[1][0]).dt.days
    df = df[streamflow[5] != "M"]
    streamflow = streamflow[streamflow[5] != "M"]
    del streamflow[5]
    streamflow.columns = ["Time", "Flow"]
    return df, streamflow

In [29]:
class Data(torch.utils.data.Dataset):
    def __init__(self, load_data, device):
        df, streamflow = load_data()
        dayl = (
            torch.from_numpy(df["dayl(s)"].to_numpy()).view(-1, 1).float().to(device)[:]
        )
        prcp = (
            torch.from_numpy(df["prcp(mm/day)"].to_numpy())
            .view(-1, 1)
            .float()
            .to(device)[:]
        )
        self.y = (
            torch.from_numpy(streamflow["Flow"].to_numpy().astype("float"))
            .view(-1, 1, 1)
            .float()
            .to(device)[:]
        )

        self.x = torch.zeros((len(df["Time"]), 1, 2)).to(device)
        self.x[:, :, 0] = dayl[:]
        self.x[:, :, 1] = prcp[:]

In [5]:
class Net(nn.Module):
    def __init__(self, nodes=51, layers=1, inputs=2, parallel=True):
        super().__init__()
        if parallel:
            self.device = "cuda:0"
            self.lstm = nn.LSTM(
                input_size=inputs, hidden_size=nodes, num_layers=layers
            ).to(self.device)
            self.lstm = nn.DataParallel(
                self.lstm, device_ids=["cuda:0", "cuda:1"], output_device=self.device
            )
            self.output_layer = nn.Linear(nodes, 1).to(self.device)

        else:
            self.device = "cuda:1"
            self.lstm = nn.LSTM(
                input_size=inputs, hidden_size=nodes, num_layers=layers
            ).to(self.device)

            self.output_layer = nn.Linear(nodes, 1).to(self.device)

    def forward(self, x):
        x, (h_n, c_n) = self.lstm(x)
        return self.output_layer(x)

In [30]:
def train(epochs, lr, parallel=True, verbose=True):
    if parallel:
        device = "cuda:0"
    else:
        device = "cuda:1"
    """dayl = torch.from_numpy(df["dayl(s)"].to_numpy()).view(-1, 1).float().to(device)[:]
    prcp = (
        torch.from_numpy(df["prcp(mm/day)"].to_numpy())
        .view(-1, 1)
        .float()
        .to(device)[:]
    )
    y = (
        torch.from_numpy(streamflow["Flow"].to_numpy().astype("float"))
        .view(-1, 1)
        .float()
        .to(device)[:]
    )

    x = torch.zeros((len(df["Time"]), 1, 2)).to(device)
    x[:, :, 0] = dayl[:]
    x[:, :, 1] = prcp[:]
    print(x.size())"""
    data = Data(load_data, device)
    model = Net(parallel=parallel)
    optimizer = torch.optim.Adam(model.parameters(), lr=2)
    loss_func = nn.MSELoss()
    model.train()
    for i in range(epochs):
        optimizer.zero_grad()
        y_pred = model.forward(data.x)
        loss = loss_func(y_pred, data.y)
        if verbose:
            print(f"Epoch {i}: {loss.item()}")
        loss.backward()
        optimizer.step()

In [31]:
train(10, 1, False, True)

Epoch 0: 5611014.0
Epoch 1: 5506238.5
Epoch 2: 5403879.0
Epoch 3: 5303975.0
Epoch 4: 5206563.5
Epoch 5: 5111678.0
Epoch 6: 5019350.5
Epoch 7: 4929607.5
Epoch 8: 4842471.5
Epoch 9: 4757964.0


In [33]:
train(10, 1, True, True)

torch.Size([12692, 1, 2])
Epoch 0: 5611925.5
Epoch 1: 5527316.5
Epoch 2: 5444274.0
Epoch 3: 5362825.5
Epoch 4: 5282994.5
Epoch 5: 5204805.5
Epoch 6: 5128277.0
Epoch 7: 5053431.0
Epoch 8: 4980284.0
Epoch 9: 4908851.5
